In [1]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
from tensorflow import keras

In [ ]:
img_path = './fig4/airplane_1.jpg'

img = cv2.imread(img_path)

In [ ]:
## dnn model import

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('model load fail')
    sys.exit

class_name = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    class_name = f.read().rstrip('\n').split('\n')
    
class_name
len(class_name)

blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                             swapRB = False)
net.setInput(blob)
prob = net.forward()
prob.shape


##
out = prob.flatten()
classid = np.argmax(out)
ClassName = class_name[classid]
confidence = out[classid]

# out.shape
text = f'{ClassName}, {confidence*100:.2f}%'

cv2.putText(img, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
            (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
## onnx

model_onnx = './googlenet/googlenet-3.onnx'
config = ''

net = cv2.dnn.readNet(model_onnx, config)

if net.empty():
    print('model load fail')
    sys.exit

class_name = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    class_name = f.read().rstrip('\n').split('\n')
    
class_name
len(class_name)

blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                             swapRB = False)
net.setInput(blob)
prob = net.forward()
prob.shape


##
out = prob.flatten()
classid = np.argmax(out)
ClassName = class_name[classid]
confidence = out[classid]

text = f'{ClassName}, {confidence*100:.2f}%'

cv2.putText(img, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
            (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
file_paths = glob.glob('./fig4/*.*')

model = './googlenet/bvlc_googlenet.caffemodel'
config = './googlenet/deploy.prototxt'
net = cv2.dnn.readNet(model, config)

class_name = []
with open('./googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    class_name = f.read().rstrip('\n').split('\n')

idx = 0

while 1:
    img = cv2.imread(file_paths[idx])
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                                swapRB = False)
    net.setInput(blob)
    prob = net.forward()

    ##
    out = prob.flatten()
    classid = np.argmax(out)
    ClassName = class_name[classid]
    confidence = out[classid]

    text = f'{ClassName}, {confidence*100:.2f}%'

    cv2.putText(img, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (0, 0, 255), 1, cv2.LINE_AA)

    cv2.imshow('img', img)
    if cv2.waitKey(1500) == 27:
        break
    
    idx += 1
    
    if idx >= len(file_paths):
        idx = 0
    
cv2.destroyAllWindows()
    

In [21]:
## 
img_path = './sunglass/sunglass.png'

img = cv2.imread(img_path)

## tensor
model_tf = './face/opencv_face_detector_uint8.pb'
config_tf = './face/opencv_face_detector.pbtxt'

face_net = cv2.dnn.readNet(model_tf, config_tf)

if face_net.empty():
    print('model load fail')
    sys.exit
    
face_blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                             swapRB = False)
face_net.setInput(face_blob)
output = face_net.forward()

detect = output[0, 0, :, :]

h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    if confidence > 0.5:
        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0 ,255), 1)
        text = f'Face, {confidence*100:.2f}%'
        cv2.putText(img, text, (x1, y1-2), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 0, 255), 1, cv2.LINE_AA)
        
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [27]:
## 
cap = cv2.VideoCapture(0)

## tensor
model_caffe = './face/res10_300x300_ssd_iter_140000_fp16.caffemodel'
config_caffe = './face/deploy.prototxt'
face_net = cv2.dnn.readNet(model_caffe, config_caffe)

if face_net.empty():
    print('model load fail')
    sys.exit
    
if not cap.isOpened():
    print('video failed')
    sys.exit
    
while 1:
    ret, frame = cap.read()
    face_blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123), swapRB = False)
    face_net.setInput(face_blob)
    output = face_net.forward()

    detect = output[0, 0, :, :]

    h, w = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        if confidence > 0.5:
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0 ,255), 1)
            text = f'Face, {confidence*100:.2f}%'
            cv2.putText(frame, text, (x1, y1-2), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255), 1, cv2.LINE_AA)
        
    # cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

In [46]:
## 
img_path = './fig2/puppy.bmp'

img = cv2.imread(img_path)

## tensor
model_tf = './face/opencv_face_detector_uint8.pb'
config_tf = './face/opencv_face_detector.pbtxt'

face_net = cv2.dnn.readNet(model_tf, config_tf)

if face_net.empty():
    print('model load fail')
    sys.exit
    
face_blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                             swapRB = False)
face_net.setInput(face_blob)
output = face_net.forward()

detect = output[0, 0, :, :]

h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    if confidence > 0.5:
        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0 ,255), 1)
        text = f'Face, {confidence*100:.2f}%'
        cv2.putText(img, text, (x1, y1-2), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 0, 255), 1, cv2.LINE_AA)
        
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [60]:
## yolo

model = './yolov7-main/yolov7/yolov3.weights'
config = './yolov7-main/yolov7/yolov3.cfg'
label = './yolov7-main/yolov7/coco.names.txt'

img = './yolov7-main/peoples.jpg'

net = cv2.dnn.readNet(model, config)

class_name = []
with open(label, 'rt') as f:
    class_name = f.read().rstrip('\n').split('\n')
    
colors = np.random.uniform(0, 255, size=(80, 3))

layers_name = net.getLayerNames()

output_layers = [layers_name[i - 1] for i in net.getUnconnectedOutLayers()]


img = cv2.imread(img)

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), mean = (0, 0, 0), swapRB = True)
net.setInput(blob)
outs = net.forward(output_layers)

h, w = img.shape[:2]

class_ids = []
confidences = []
boxes = []

confidence_thresh = 0.5
nms_thresh = 0.4

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > confidence_thresh:
            cx = int(detection[0] * w)
            cy = int(detection[1] * h)
            bw = int(detection[2] * w)
            bh = int(detection[3] * h)
            
            sx = int(cx - bw / 2)
            sy = int(cy - bh / 2)
            
            boxes.append([sx, sy, bw, bh])
            confidences.append(confidence)
            class_ids.append(class_id)
            
indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, nms_thresh)

for i in indices:
    sx, sy, bw, bh = boxes[i]
    label = f'{class_name[class_ids[i]]}, {confidences[i]*100:.2f}'
    color = colors[class_ids[i]]
    cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
    cv2.putText(img, label, (sx, sy - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                color, 2, cv2.LINE_AA)



cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()


In [61]:
cap = cv2.VideoCapture(0)


## yolo

model = './yolov7-main/yolov7/yolov3.weights'
config = './yolov7-main/yolov7/yolov3.cfg'
label = './yolov7-main/yolov7/coco.names.txt'

net = cv2.dnn.readNet(model, config)

class_name = []
with open(label, 'rt') as f:
    class_name = f.read().rstrip('\n').split('\n')
    
colors = np.random.uniform(0, 255, size=(80, 3))

layers_name = net.getLayerNames()

output_layers = [layers_name[i - 1] for i in net.getUnconnectedOutLayers()]



while 1:
    ret, frame = cap.read()
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), mean = (0, 0, 0), swapRB = True)
    net.setInput(blob)
    outs = net.forward(output_layers)

    h, w = frame.shape[:2]

    class_ids = []
    confidences = []
    boxes = []

    confidence_thresh = 0.5
    nms_thresh = 0.4

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_thresh:
                cx = int(detection[0] * w)
                cy = int(detection[1] * h)
                bw = int(detection[2] * w)
                bh = int(detection[3] * h)
                
                sx = int(cx - bw / 2)
                sy = int(cy - bh / 2)
                
                boxes.append([sx, sy, bw, bh])
                confidences.append(confidence)
                class_ids.append(class_id)
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, nms_thresh)

    for i in indices:
        sx, sy, bw, bh = boxes[i]
        label = f'{class_name[class_ids[i]]}, {confidences[i]*100:.2f}'
        color = colors[class_ids[i]]
        cv2.rectangle(frame, (sx, sy, bw, bh), color, 2)
        cv2.putText(frame, label, (sx, sy - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    color, 2, cv2.LINE_AA)




    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()